## <center> Image recognition </center>
### <center> Train your own model vs Transfer learning </center>

In [55]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import to_categorical, load_img
from keras.callbacks import TensorBoard
from keras.applications import vgg16, resnet
from pathlib import Path

### I. Loading data

In [17]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [12]:
print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))

X_train shape: (50000, 32, 32, 3)
X_test shape: (10000, 32, 32, 3)


- CIFAR10 is 60K image split into 50K on train and 10 on test. Each image size is: 32 x 32 x 3

### II. Preprocessing images

#### 1. Normalisation

First, let's convert our data into float variables

In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Next, let's divide by 255 to normalize data between 0 & 1

In [6]:
X_train /= 255
X_test /= 255

#### 2. Convert labels to categorical

In [13]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

### III. Trained from scratch model

#### 1. Model creation

In [30]:
model = Sequential()

# First convolution block
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolution block
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolution block
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten then dense layer
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(10, activation="softmax"))

# Model compilation
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

# Tensorboard logging
logger = TensorBoard(log_dir='logs/3Conv_blocks', write_graph=True, histogram_freq=3)

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 32)        9248      
                                                      

#### 2. Training the model

In [32]:
model.fit(
    X_train,
    y_train,
    shuffle=True,
    batch_size=32,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[logger]
)

Epoch 1/30
1563/1563 [==============================] - 41s 24ms/step - loss: 1.6862 - accuracy: 0.3730 - val_loss: 1.3368 - val_accuracy: 0.5086
Epoch 2/30
1563/1563 [==============================] - 40s 26ms/step - loss: 1.3113 - accuracy: 0.5258 - val_loss: 1.1875 - val_accuracy: 0.5671
Epoch 3/30
1563/1563 [==============================] - 42s 27ms/step - loss: 1.1673 - accuracy: 0.5820 - val_loss: 1.0401 - val_accuracy: 0.6371
Epoch 4/30
1563/1563 [==============================] - 41s 26ms/step - loss: 1.0709 - accuracy: 0.6189 - val_loss: 0.9487 - val_accuracy: 0.6663
Epoch 5/30
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0093 - accuracy: 0.6409 - val_loss: 0.8926 - val_accuracy: 0.6870
Epoch 6/30
1563/1563 [==============================] - 49s 31ms/step - loss: 0.9640 - accuracy: 0.6590 - val_loss: 0.8352 - val_accuracy: 0.7073
Epoch 7/30
1563/1563 [==============================] - 44s 28ms/step - loss: 0.9208 - accuracy: 0.6737 - val_loss: 0.8316 -

#### 3. Saving the model's structure

In [33]:
model_structure = model.to_json()
f = Path('model_structure.json')
f.write_text(model_structure)

5872

#### 4. Saving the model's weights

In [34]:
model.save_weights('model_weights.h5')

#### 5. Loading the model

- Loading the model's structure

In [35]:
p = Path('model_structure.json')
model_structure = p.read_text()

In [36]:
model = model_from_json(model_structure)

- Loading the model parameters

In [37]:
model.load_weights('model_weights.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 6. Model evaluation

In [38]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.6620 - accuracy: 0.7720


[0.662005603313446, 0.7720000147819519]

#### 7. Model prediction

- Loading external images to test

In [8]:
frog = load_img("test_images/frog.png", target_size=(32, 32))
bay = load_img("test_images/bay.jpg", target_size=(32, 32))
dog = load_img("test_images/dog.png", target_size=(32, 32))

In [9]:
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

- Predicting the 3 test image's class

In [41]:
pred = model.predict(np.stack([frog, bay, dog], axis=0))
pred_classes = []
for prob in pred:
    most_likely = int(np.argmax(prob))
    class_label = class_labels[most_likely]
    pred_classes.append(class_label)
print(pred_classes)

1/1 [==============================] - 0s 234ms/step
['Frog', 'Car', 'Car']


### IV. Transfer Learning - VGG16

#### 0. Reading data

In [72]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

#### 1. Pretrained VGG16 without output layer

In [73]:
pretrained_model = vgg16.VGG16(weights="imagenet",
                               include_top=False,
                               input_shape=(32, 32, 3))

#### 2. Preprocessing data

In [74]:
X_train = vgg16.preprocess_input(X_train)
X_test = vgg16.preprocess_input(X_test)

#### 3. Extracting images fautures

In [75]:
features_train = pretrained_model.predict(X_train)
features_test = pretrained_model.predict(X_test)

313/313 [==============================] - 60s 185ms/step


#### 3. Fine-tuning the model to our use case

In [78]:
model = Sequential()
model.add(Flatten(input_shape=features_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [79]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Tensorboard logging
logger = TensorBoard(log_dir='logs/fined-tuned-vgg16', write_graph=True, histogram_freq=3)

In [80]:
model.fit(features_train,
          y_train,
          validation_data = (features_test, y_test),
          epochs=15,
          callbacks = [logger],
          shuffle=True)

Epoch 1/15
1563/1563 [==============================] - 9s 4ms/step - loss: 2.4413 - accuracy: 0.4611 - val_loss: 1.1856 - val_accuracy: 0.5855
Epoch 2/15
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2865 - accuracy: 0.5595 - val_loss: 1.0965 - val_accuracy: 0.6156
Epoch 3/15
1563/1563 [==============================] - 4s 3ms/step - loss: 1.2033 - accuracy: 0.5857 - val_loss: 1.0596 - val_accuracy: 0.6403
Epoch 4/15
1563/1563 [==============================] - 5s 3ms/step - loss: 1.1632 - accuracy: 0.6042 - val_loss: 1.0461 - val_accuracy: 0.6473
Epoch 5/15
1563/1563 [==============================] - 5s 3ms/step - loss: 1.1277 - accuracy: 0.6187 - val_loss: 1.0518 - val_accuracy: 0.6479
Epoch 6/15
1563/1563 [==============================] - 5s 3ms/step - loss: 1.1086 - accuracy: 0.6247 - val_loss: 1.0427 - val_accuracy: 0.6512
Epoch 7/15
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0852 - accuracy: 0.6285 - val_loss: 1.0404 - val_accuracy:

### IV. Transfer Learning - Resnet50

#### 0. Reading data

In [81]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

#### 1. Pretrained ResNet50 without output layer

In [82]:
pretrained_model = resnet.ResNet50(weights="imagenet",
                               include_top=False,
                               input_shape=(32, 32, 3))

#### 2. Preprocessing data

In [83]:
X_train = resnet.preprocess_input(X_train)
X_test = resnet.preprocess_input(X_test)

#### 3. Extracting images fautures

In [84]:
features_train = pretrained_model.predict(X_train)
features_test = pretrained_model.predict(X_test)

313/313 [==============================] - 34s 97ms/step


#### 3. Fine-tuning the model to our use case

In [85]:
model = Sequential()
model.add(Flatten(input_shape=features_train.shape[1:]))
model.add(Dense(528, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [86]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Tensorboard logging
logger = TensorBoard(log_dir='logs/fined-tuned-resnet50', write_graph=True, histogram_freq=3)

In [87]:
model.fit(features_train,
          y_train,
          validation_data = (features_test, y_test),
          epochs=15,
          callbacks = [logger],
          shuffle=True)

Epoch 1/15
1563/1563 [==============================] - 17s 9ms/step - loss: 1.5684 - accuracy: 0.4824 - val_loss: 1.1511 - val_accuracy: 0.6166
Epoch 2/15
1563/1563 [==============================] - 13s 8ms/step - loss: 1.2582 - accuracy: 0.5698 - val_loss: 1.0787 - val_accuracy: 0.6385
Epoch 3/15
1563/1563 [==============================] - 13s 8ms/step - loss: 1.1951 - accuracy: 0.5981 - val_loss: 1.0592 - val_accuracy: 0.6426
Epoch 4/15
1563/1563 [==============================] - 14s 9ms/step - loss: 1.1509 - accuracy: 0.6112 - val_loss: 1.0396 - val_accuracy: 0.6509
Epoch 5/15
1563/1563 [==============================] - 14s 9ms/step - loss: 1.1153 - accuracy: 0.6218 - val_loss: 1.0364 - val_accuracy: 0.6456
Epoch 6/15
1563/1563 [==============================] - 13s 9ms/step - loss: 1.0923 - accuracy: 0.6324 - val_loss: 1.0190 - val_accuracy: 0.6515
Epoch 7/15
1563/1563 [==============================] - 14s 9ms/step - loss: 1.0705 - accuracy: 0.6421 - val_loss: 1.0264 - val_ac